# TensorFlow Lite Micro - LSTM example

The code cells below walk you through building a LSTM network for classifying MNIST data. *This is currently only possible in Python 3.10.x with TensorFlow 2.15.x*. Find the steps listed below:

- The software / library versions are checked
- The dataset is loaded and normalised
- The TensorFlow model is built using one LSTM layer and one Dense layer
- The TensorFlow model is trained on the training data, and verified using the validation data
- The Input and Output layers of the model are converted into a fixed shape (as per https://www.tensorflow.org/lite/models/convert/rnn#keras_lstm_conversion_recommended)
- The fixed shape model is converted using the TensorFlow Lite converter and saved
- The TensorFlow Lite model is converted into a C-array header file

---

You can edit any part of the dataset loading and model as you like. It is also possible to add code for visualising the model training history.

To add files (e.g. your own dataset), open the Colab File Browser by clicking the 📂 (Folder icon on the left). You can drag and drop files in this window to upload, or click the ⬆ (Upload files button). *Files will not be perisistant in this storage, they are deleted after the Google runtime is disconnected. Back up your files often and carefully!*

In [ ]:
# Check Python version
import sys
print(f"Python version (should be 3.10.x): {sys.version}")

Python version (should be 3.10.x): 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
# Import libraries
import tensorflow as tf
import numpy as np
import sklearn
print(f"TensorFlow version (should be 2.15.x): {tf.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

TensorFlow version (should be 2.15.x): 2.15.0
Numpy version: 1.25.2
Scikit-learn version: 1.2.2


In [ ]:
# Setup the dataset, using MNIST digit data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values to 0-1
x_train = x_train / 255.
x_test = x_test / 255.

# Set the data up as numpy floats
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [ ]:
# Create the TensorFlow LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=x_train[0].shape, name="input"),
    tf.keras.layers.LSTM(20, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax", name="output")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
    )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 20)            3920      
                                                                 
 flatten (Flatten)           (None, 560)               0         
                                                                 
 output (Dense)              (None, 10)                5610      
                                                                 
Total params: 9530 (37.23 KB)
Trainable params: 9530 (37.23 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model using the data
model.fit(
    x_train,
    y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    batch_size=32
)

Epoch 1/5
1875/1875 [==============================] - 32s 14ms/step - loss: 0.3366 - accuracy: 0.9010 - val_loss: 0.1436 - val_accuracy: 0.9562
Epoch 2/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1209 - accuracy: 0.9633 - val_loss: 0.1105 - val_accuracy: 0.9631
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0888 - accuracy: 0.9732 - val_loss: 0.0761 - val_accuracy: 0.9744
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0720 - accuracy: 0.9773 - val_loss: 0.0712 - val_accuracy: 0.9771
Epoch 5/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0620 - accuracy: 0.9805 - val_loss: 0.0610 - val_accuracy: 0.9794


In [ ]:
# Convert the model to have fixed input and output shapes
fixed_input = tf.keras.layers.Input(
    shape=x_train[0].shape,
    batch_size=1,
    dtype=model.inputs[0].dtype,
    name="fixed_input"
    )
fixed_output = model(fixed_input)

static_model = tf.keras.models.Model(fixed_input, fixed_output)

In [ ]:
# Save the model as TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(static_model)
tflite_model = converter.convert()

open("model.tflite", "wb").write(tflite_model)

41240

In [ ]:
# Convert the TensorFlow Lite file to a C-array header file
def convert_tflite_to_c(tflite_path: str = 'model.tflite', model_name: str = 'model') -> str:
    """
    Converts TFLite models into C-compatible header files for Arduino etc.

    Parameters
    ----------
    tflite_path : str
        Path to the TFLite model file
        Default path is 'model.tflite'
    model_name : str
        Output model name
        Default value is 'model'

    Returns
    -------
    model_path : str
        Path to the converted C-compatible model

    Raises
    ------
    ValueError
        If the provided model is not a TFLite model
    """

    # Check if file is a tflite model. If not, raise ValueError.
    if not tflite_path.endswith('.tflite'):
        raise ValueError("The provided file is not a TFLite model.")

    # Open the TFLite model file in binary mode and read its content into 'tflite_content'.
    with open(tflite_path, 'rb') as tflite_file:
        tflite_content = tflite_file.read()

    # Calculate the length of 'tflite_content' (i.e., the size of the TFLite model in bytes).
    array_length = len(tflite_content)

    # Split 'tflite_content' into chunks of 12 bytes each and convert each chunk to a hexadecimal string.
    # This is done so that the TFLite model can be represented as an array in C-compatible format.
    hex_lines = [', '.join([f'0x{byte:02x}' for byte in tflite_content[i:i + 12]]) for i in
                 range(0, len(tflite_content), 12)]

    # Join the chunks of hexadecimal strings with newlines to format them neatly.
    hex_array = ',\n     '.join(hex_lines)

    # Open a header file in write mode and write out the TFLite model as an array.
    with open(model_name + '.h', 'w') as header_file:
        # Write the length of the TFLite model to the header file.
        header_file.write(f'unsigned int {model_name}_len = {array_length};\n\n')

        # Write out the TFLite model as an array in C-compatible format.
        header_file.write(f'alignas(8) const unsigned char {model_name}[] = {{\n     ')
        header_file.write(f'{hex_array}\n')
        header_file.write(f'}};\n')

    # Return the name of the generated header file.
    return model_name + '.h'

convert_tflite_to_c('model.tflite', 'lstm_model')

'lstm_model.h'

In order to get the C-array header model file, open the Colab File Browser by clicking the 📂 (Folder icon on the left) and clicking the three dots, the select "Download".